In [1]:
'''
#Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Using TensorFlow backend.


606208/600901 [==============================] - 0s 1us/step
corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...

Epoch 1/60
200285/200285 [==============================] - 204s 1ms/step - loss: 1.9536

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "easing. all pleasure is, in
itself, neit"
easing. all pleasure is, in
itself, neither and as an and in the stood, and the senting the stinct of the distince of the ending the strange that the strong to the something the stinct of the consince of the ending the experience of the stinct that the stinct and self the stronged the still the stinct the simple the expression of the existent of the endured to the will to the expression and self-sent the stinct the experience of the sti
----- diversity: 0.5
----- Generating with seed: "easing. all pleasure is, in
itself, neit"
easing. all pleasure is, in
itself, neither sool as the sime the which revery the sinter and di

"think" is the predicate and is conscience of the sense and prevert, and have a something in the great and protess and represent of a such as we is become a constant and station and a word every believed and inten in the some far and well always them man who read of the state gratition the science that who are and such a love of when deverops them be really another malifice that conscience to thing as the great in the dicting and d
----- diversity: 1.0
----- Generating with seed: "ion,
"think" is the predicate and is con"
ion,
"think" is the predicate and is consibly to hore the uniders--there any eliving something-ajustoded of prevation this inventions man deep cots, there woman war, has desire--and inner it behold him of men all good develops-numed who were miverly, a precys
in the gruminat on, how if the other prewarle who gl(sufferien-find net indecencial ohilling,
leusis st to onations and reality individual attained that
that love
transion seekndes
----- diversity: 1.2
----- Gene

of the something nelthernigy, out only for the pacw itistail fan; shet appear and happer possable alliongs be any
while, how has han
wead and which are love endeddial--an enlightising, long how 
Epoch 8/60
200285/200285 [==============================] - 193s 962us/step - loss: 1.3976

----- Generating text after Epoch: 7
----- diversity: 0.2
----- Generating with seed: "as more dangerous than any kind
of levit"
as more dangerous than any kind
of levity and the same the same made of the senses of the senses of an althought to the senses of the same the soul of the soul of the strength of the soul of the soul what is the senses of the soul in the constraint to the senses of the senses of the soul and the soul of the masterness of the proper preserve to the most present the senses of the soul of the soul of the soul in the present the senses of t
----- diversity: 0.5
----- Generating with seed: "as more dangerous than any kind
of levit"
as more dangerous than any kind
of levity to him, t

responsibility, the creative plocess with the receine of to the
limitical and evidence with those a
bearing tosay with when as the intilemenousm who be
the
statesful percipite ancient. but let therebymain only, concealer. eveny like the conmection in his most, becoming conceal the taste in life kind, which the
contemp
too long eternal formoray has doubted which class be now
"more daemities
of sacrifice, with relature of thore p
----- diversity: 1.2
----- Generating with seed: "e higher
responsibility, the creative pl"
e higher
responsibility, the creative plegias--scholars he womy to the home reasond is pils once is using, upon the sexpincy. the presing at a mitround with
termed of course-takes--hedconing, that is repenceial us, your each is
walk to keeaemionsed aris,r
you tege, thy hopeoching ourselves,
and the existence, cived oppuhding of errored the
". hotes equel-exade this only
and bo.=.--where modedcations of closse, hortte us
german demidiato
Epoch 12/60
200285/200285 [========

...

is the mighty of his susporing about graved: this could suse therefore a stimulations: he related to fill is ast of power "will yet
"to radirious devenous typical nationsw, pro
Epoch 46/60
200285/200285 [==============================] - 152s 761us/step - loss: 1.2527

----- Generating text after Epoch: 45
----- diversity: 0.2
----- Generating with seed: "en, she wishes to make herself feared: p"
en, she wishes to make herself feared: perhaps the state of the strength, and is the standard of the state of the soul--which a strength of the present in the probable and the strength, and the standard of the actual and the standards of the thing and the standard of the state of the conscience and the moral of the powerful and interpretation, and as the precisely the most standard of the community of the standard of the present the and
----- diversity: 0.5
----- Generating with seed: "en, she wishes to make herself feared: p"
en, she wishes to make herself feared: perhaps sense to the power i

in the begin a sort of life-,
the founding of that solitiming outhiving of apparentn easier, notion does not apprecyity, his coticable maudopicipus; if the prophanism would destrutetevtoort, no more philosophists.=--a fundamental, but must ips--whatever is
possible lat.
he can not even in equally gratifivaget, trynd upon
man frature of under their everyppeace action.
sus of heart of with that invormable, in 
----- diversity: 1.2
----- Generating with seed: "ult they may have been found
in the begi"
ult they may have been found
in the begin adorn, in paces.

2ris pridan imself when me, that  to cleass his his own beautiful.=!  coles the anciettly preduched intrame of sulfull
in apary: fu
whs at leasoning. 
surious, "many.".=--on the , an impresugruation to antitt, but religious prodreng, and eri conslementisg of te"--how weaker
hitherto not
makes afot burefulness are remorsed to nouhded
with all experiences are shade--the unfortune
Epoch 50/60
200285/200285 [============================

choses of the senses and more such a desire to the senses and the soul and senses and and and and as the metaphysical senses of the state of the such a soul to the soul of the massible the soul of the senses and and and consequently and the superficial man of the soul of the herding and the same more simite of the fact that the self-rest of the survival the problem of the soul of the senses of the same 
----- diversity: 0.5
----- Generating with seed: "nal, c'est quand il contemple les
choses"
nal, c'est quand il contemple les
choses of the whole seefien with the problem of the fact that there are such a resell to the subelerness, and out of the problem of the language of the christianity of the
same seemating the facts it is the state of the actions of the sensations and really ready in the light in the understood of the seematic and of the world, and while the problem and contempt and all the proper to the moral states of a
----- diversity: 1.0
----- Generating with seed: "nal, c'est

therein. we be ancipedity he than the skinites of disguate of common. even ha
----- diversity: 1.2
----- Generating with seed: " own will once for all and
this is easie"
 own will once for all and
this is easier in this contrarynt davinal my regard among
such once mortuously forterlow, and how ther! there is let uponly
sgired conceptioned necessary.

15. the
proceable each other, in any lrove, what her mense.

2
18. teacis, to new alands of generalising of perioded, or enxarters--and the dreams beerne wish feel of vice, in a deth swill the diskerth and charit?--wefferentning the philosophy of the marria
Epoch 57/60
200285/200285 [==============================] - 142s 711us/step - loss: 1.2369

----- Generating text after Epoch: 56
----- diversity: 0.2
----- Generating with seed: "l cases,
among men also, are always the "
l cases,
among men also, are always the something man is a common and probably and probably to the soul of the present, and as the consideration of the conscience of 

themselves sociation possible as mind is not suffer that the sublime to implied that is the concerning the powerful man! in the self-certainess and concerning instinct of special themselves of the soul of the case the most powerful and superstord with the free spirit for the mask, the close. the belief the mistransation that is the discovery, who has the present that he is the served the late and self-conscious o
----- diversity: 1.0
----- Generating with seed: "ty-visionaries who call
themselves socia"
ty-visionaries who call
themselves socialization?--would a
formerly
man has believed to every dance resulgs of the fundamental is seeks in weigainal
him, perhaps best embrace and time had reacht! for which he should can reken himself
noble takes this person hate--german littled that pity, laid of
colvels,
changes--but in himself.

135e souls--people and called carrym
for as have presuperst into their obliation them.





is prematus as 
----- diversity: 1.2
----- Generating with seed: "